## Department of Computer Science, University of York
### DATA: Introduction to Data Science

## Task 1: Domain Analysis  (5 marks)

Given the business domain and the data overview presented (in the assessment paper), provide a brief description of

* the business problem and its significance to the relevant sector;
* the link between the business problem and the field of data science;
* the main areas of investigation; and
* potential ideas and solutions.


**Word Limit:** 300 words

**Write your answer here (text cell(s) to be used, as appropriate)**

The questions at hand are of demographics, loan default prediction, and expansion area. Banks like this one make profit from interest on loans and credit cards[^1], so the objectives are to refuse loans that will default, give high-value customers reasons to continue doing business with the bank, and onboard new customers.

We can leverage data science to load and quickly perform operations on huge amounts of figures and categorical data (e.g. over 100,000 rows of transactions), creating descriptions and visualisations of the data that people can understand. We can inform decision making by using techniques from statistics to summarise business operations and make predictions of the outcomes of providing credit.

The investigations I have chosen to carry out will be the coverage of the customer base over different cities, and the way a customer's characteristics and prior usage of services relate to credit outcomes. Proposals will include strategies for predicting whether to authorise products, how to identify customers to target with perks, and potential geographic areas of expansion to focus on, using models like logistic regression and visualisation like bar charts.

[^1]: https://www.investopedia.com/terms/c/commercialbank.asp

In [1]:
### Write your answer here (code cell(s) to be used, as appropriate)



----
----


## Task 2: Database Design (25 marks)


Having understood the business domain, present a conceptual design in the form of an entity-relationship (ER) model that would be helpful in creating a database for the bank.

The bank data currently exists in the form of a csv file called *BankRecords.csv*, provided on VLE (path given in page 5, assessment paper). This file has all the existing records. The table available in the csv file is unnormalised. The information about its different columns is given in Tables 1 and 2 (in the assessment paper).

Following the standard principles of database normalisation, normalise the given table (*BankRecords.csv*) to a database schema that has minimum redundancies. Then, using the designed schema, create an SQLite database.

Your answer should include the SQL statements needed to accomplish this step. Your submission should also include the created SQLite database file.

Your answer should clearly cover the following:
* Any assumptions you are making about the given scenario;
* The designated keys, existing relationships, and identified functional dependencies;
* The steps followed and justifications for the decisions made.

**World Limit**: 500 words. This limit applies only to the explanations. There is no limit on any associated code/SQL statements or figures.

**Write your answer here (text cell(s) to be used, as appropriate)**

# Normal form

The data is already in first normal form as there are no composite values, so we can continue to 2NF.

This table is clearly not in second normal form. Loan date is dependent on loan ID, name is dependent on client ID, etc. etc. There are a bunch of values that should be primary keys all in the same table. I split up the dataset into eight separate tables intuitively based on the "Data Overview" and "Description of columns" from the specification. Each unique ID gets its own table.

Now in 2NF, I noticed noticed a transitive dependency - I had included dispID, clientID, and accountID together in their relevant tables. I realised clientID and accountID are both just dependent on dispID. Therefore I removed them from all the tables except Dispositon. Finally, we are in third normal form and finished.

I briefly thought there was a transitive dependency from "region" to "noAreas" because the data overview said noAreas was the number of *cities* in the *region*, and was repeated for each *city*, but the inconsistent noAreas for each region and the mismatch of actual city count implies this is an error in the data overview.

Reference:
- DATA-Lecture4-DatabaseNormalisationAndSQL.pdf
- https://en.wikipedia.org/wiki/Database_normalization

# ERM
For the ERM diagram, see `DATA Essay.jpg`.

![ERM Diagram](DATA%20Essay.jpg)

Reference:
- DATA-Lecture3-RelationalDatabases.pdf
- DATA-Practical3Part2 - Mapping ER model to tables - Solution.pdf
- https://en.wikipedia.org/wiki/Entity%E2%80%93relationship_model

In [2]:
import sqlite3
from pathlib import Path

In [3]:
### Write your answer here (code cell(s) to be used, as appropriate)

# the following classifier fields could be made integers:
#   Account
#     statementFrequency
#   Loan
#     loanStatus
#   StandingOrder
#     NOT paymentType
#   BankTransaction
#     transType
#     operation
#     NOT transDetail
#   Disposition
#     dispType
#   CreditCard
#     cardType

sql_script_path = Path("create_db.sqlite3")
sqlf = Path("BankRecords.db")


def create_db():
    sql_create = sql_script_path.read_text(encoding="utf-8")
    
    print("Deleting database")
    try:
        sqlf.unlink()
    except FileNotFoundError:
        print("Database didn't exist")
    else:
        print("Deleted database")
    
    print("Creating database")
    con = sqlite3.connect(sqlf)
    with con:
        con.executescript(sql_create)
    con.close()
    print("Done")


create_db()

Deleting database
Deleted database
Creating database
Done


In [4]:
import pandas as pd

In [5]:
%%time

# load dataframe
DATE_FORMAT_CSV = "%y%m%d"

csvf = Path("BankRecords.csv")

print("Loading csv")

# could manually convert to datetime after reading
# to set the unit to something better for dates than ns
# but this is fine
date_columns = [
    "creation_date",
    "loan_date",
    "trans_date",
    "card_issued",
]
date_formats = {c: DATE_FORMAT_CSV for c in date_columns}
date_formats["card_issued"] = f"{DATE_FORMAT_CSV} 00:00:00"  # they lied about this one

df = pd.read_csv(
    csvf,
    parse_dates=date_columns,
    date_format=date_formats,
    low_memory=False,
)

Loading csv
CPU times: user 6.54 s, sys: 4.93 s, total: 11.5 s
Wall time: 12.3 s


In [6]:
df.head()

,account_id,frequency,creation_date,loan_id,loan_date,loan_amount,loan_duration,loan_payments,loan_status,order_id,...,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16
0,1,Monthly,1995-03-24,NaN,NaT,NaN,NaN,NaN,NaN,29401.0,...,2,1,4,65.3,8968,2.83,3.35,131,1740.0,1910
1,1,Monthly,1995-03-24,NaN,NaT,NaN,NaN,NaN,NaN,29401.0,...,2,1,4,65.3,8968,2.83,3.35,131,1740.0,1910
2,1,Monthly,1995-03-24,NaN,NaT,NaN,NaN,NaN,NaN,29401.0,...,2,1,4,65.3,8968,2.83,3.35,131,1740.0,1910
3,1,Monthly,1995-03-24,NaN,NaT,NaN,NaN,NaN,NaN,29401.0,...,2,1,4,65.3,8968,2.83,3.35,131,1740.0,1910
4,1,Monthly,1995-03-24,NaN,NaT,NaN,NaN,NaN,NaN,29401.0,...,2,1,4,65.3,8968,2.83,3.35,131,1740.0,1910


In [7]:
# note that some columns we would expect to be int are float instead
# because int can't be NaN but float can
print(df.shape)
print(df.dtypes)
df.columns

(2243458, 46)
account_id                  int64
frequency                  object
creation_date      datetime64[ns]
loan_id                   float64
loan_date          datetime64[ns]
loan_amount               float64
loan_duration             float64
loan_payments             float64
loan_status                object
order_id                  float64
bank_to                    object
account_to                float64
order_amount              float64
payment_type               object
trans_id                    int64
trans_date         datetime64[ns]
trans_type                 object
operation                  object
trans_amount              float64
balance                   float64
trans_detail               object
partner_bank               object
partner_account           float64
disp_id                     int64
client_id                   int64
disp_type                  object
card_id                   float64
card_type                  object
card_issued        datetime64[ns]


Index(['account_id', 'frequency', 'creation_date', 'loan_id', 'loan_date',
       'loan_amount', 'loan_duration', 'loan_payments', 'loan_status',
       'order_id', 'bank_to', 'account_to', 'order_amount', 'payment_type',
       'trans_id', 'trans_date', 'trans_type', 'operation', 'trans_amount',
       'balance', 'trans_detail', 'partner_bank', 'partner_account', 'disp_id',
       'client_id', 'disp_type', 'card_id', 'card_type', 'card_issued',
       'birth_number', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9',
       'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16'],
      dtype='object')

In [49]:
x = "loan_amount"
print(df[(df[x] % 1 != 0) & df[x].notna()])

Empty DataFrame
Columns: [account_id, frequency, creation_date, loan_id, loan_date, loan_amount, loan_duration, loan_payments, loan_status, order_id, bank_to, account_to, order_amount, payment_type, trans_id, trans_date, trans_type, operation, trans_amount, balance, trans_detail, partner_bank, partner_account, disp_id, client_id, disp_type, card_id, card_type, card_issued, birth_number, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16]
Index: []


In [40]:
pd.set_option("display.max_columns", 100)
print(df[df["trans_id"].notna() & df["card_id"].notna()].tail())

         account_id frequency creation_date  loan_id  loan_date  loan_amount  \
2241480       11359   Monthly    1994-10-01   7305.0 1996-08-06      54024.0   
2241481       11359   Monthly    1994-10-01   7305.0 1996-08-06      54024.0   
2241482       11359   Monthly    1994-10-01   7305.0 1996-08-06      54024.0   
2241483       11359   Monthly    1994-10-01   7305.0 1996-08-06      54024.0   
2241484       11359   Monthly    1994-10-01   7305.0 1996-08-06      54024.0   

         loan_duration  loan_payments loan_status  order_id bank_to  \
2241480           12.0         4502.0           A   46329.0      OP   
2241481           12.0         4502.0           A   46329.0      OP   
2241482           12.0         4502.0           A   46329.0      OP   
2241483           12.0         4502.0           A   46329.0      OP   
2241484           12.0         4502.0           A   46329.0      OP   

         account_to  order_amount payment_type  trans_id trans_date  \
2241480  33080721.0  

In [8]:
# since a 2-digit year is ambiguous we have to check this works fine
# python's datetime pivot year is 1970
# so we can verify all the years are between 70 and 99 like so:
for c in date_columns:
    print(c, sum(df[c] > pd.Timestamp(year=2000, month=1, day=1)))

# therefore, in the special parsing we do for birth_number
# we can assume all dates lie in the decade 1900

creation_date 0
loan_date 0
trans_date 0
card_issued 0


In [6]:
%%time

# ISO 8601 date
# https://www.sqlite.org/lang_datefunc.html
DATE_FORMAT_SQL = "%Y-%m-%d"


# reset db for testing
create_db()


def filter_df(columns: list[str]):
    return df.filter(columns).groupby(columns[0]).first()


print("Inserting data")
con = sqlite3.connect(sqlf)
cur = con.cursor()

accounts = filter_df(
    ["account_id", "frequency", "creation_date"]
)
# no inplace option for this 3':
accounts["creation_date"] = accounts["creation_date"].dt.strftime(DATE_FORMAT_SQL)
cur.executemany(
    "INSERT INTO Account VALUES (?, ?, ?);",
    accounts.itertuples()
)

loans = filter_df(
    ["loan_id", "disp_id", "loan_date", "loan_amount", "loan_duration", "loan_payments", "loan_status"]
)
loans["loan_date"] = loans["loan_date"].dt.strftime(DATE_FORMAT_SQL)
cur.executemany(
    "INSERT INTO Loan VALUES (?, ?, ?, ?, ?, ?, ?);",
    loans.itertuples()
)

standing_orders = filter_df(
    ["order_id", "disp_id", "bank_to", "account_to", "order_amount", "payment_type"]
)
cur.executemany(
    "INSERT INTO StandingOrder VALUES (?, ?, ?, ?, ?, ?);",
    standing_orders.itertuples()
)

transactions = filter_df(
    [
        "trans_id", "disp_id",
        "trans_date", "trans_type", "operation", "trans_amount", "balance", "trans_detail", "partner_bank", "partner_account",
    ]
)
transactions["trans_date"] = transactions["trans_date"].dt.strftime(DATE_FORMAT_SQL)
cur.executemany(
    "INSERT INTO BankTransaction VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);",
    transactions.itertuples()
)

clients = filter_df(
    ["client_id", "a1", "birth_number"]
)
# separating birth number into relevant columns
# could also do this as string operations
# bool mask of all rows with MM+50
clients["gender"] = ((clients["birth_number"] // 100) % 100) > 12
# remove offset
# 100(x + 50) = 100x + 5000
clients.loc[clients["gender"], "birth_number"] -= 5000
# parse date
clients["birth_number"] += 19000000  # see above cell
clients["birth_number"] = pd.to_datetime(clients["birth_number"], format="%Y%m%d")
clients["birth_number"] = clients["birth_number"].dt.strftime(DATE_FORMAT_SQL)
cur.executemany(
    "INSERT INTO Client VALUES (?, ?, ?, ?);",
    clients.itertuples()
)

dispositions = filter_df(
    ["disp_id", "account_id", "client_id", "disp_type"]
)
# dispo elysium
cur.executemany(
    "INSERT INTO Disposition VALUES (?, ?, ?, ?);",
    dispositions.itertuples()
)

credit_cars = filter_df(
    ["card_id", "disp_id", "card_type", "card_issued"]
)
credit_cars["card_issued"] = credit_cars["card_issued"].dt.strftime(DATE_FORMAT_SQL)
cur.executemany(
    "INSERT INTO CreditCard VALUES (?, ?, ?, ?);",
    credit_cars.itertuples()
)

cities = filter_df(
    [f"a{i}" for i in range(1, 16+1)]
)
cur.executemany(
    "INSERT INTO City VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);",
    cities.itertuples()
)

cur.close()
con.commit()
con.close()
print("Inserting data complete")

Deleting database
Deleted database
Creating database
Done
Inserting data
Inserting data complete
CPU times: user 8.62 s, sys: 310 ms, total: 8.93 s
Wall time: 9.22 s


----
----


## Task 3: Research Design (25 Marks)

Using the database designed in Task 2, design and implement **five** potential modelling solutions to achieve the aim of the Data Intelligence team. You need to provide clear justifications about the techniques selected in the context of the 'problem in hand'. Your design must consist of a combination of inferential statistics, supervised learning algorithms, and unsupervised learning algorithms, and include **at least one** of those techniques. Finally, your modelling solutions should be of sufficient complexity, combining information from multiple tables from the database built in Task 2, as appropriate. Your answer should clearly show the queries made to the database. If amendments are made to the database, the commands should be clearly included in your answer.

Your answer should clearly cover the following:
* Any assumptions you are making about the given scenario;
* Any data processing and data integrity steps you would undertake to make the data fit for purpose;
* Which technique(s) you would apply for each solution and why;
* An evaluation of the techniques applied in terms of the accuracy of their results (or any other suitable evaluation measure);
* Algorithmic parameters should be adequately stated and discussed;
* A discussion of ethical considerations arising from the solutions selected.

**World Limit**: 500 words. This limit applies only to the explanations. There is no limit on any associated code or figures.

**Write your answer here (text cell(s) to be used, as appropriate)**

techniques
1. inferential - standard distribution and hypothesis testing
2. supervised - linear regression (simple, multiple, polynomial)
    - https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html
3. supervised - decision tree (classification)
4. unsupervised - clustering (classification)

suggestions from paper
- customer risk assessment
- customer retention scheme

idea 1. logistic regression on loan A/B with city characteristics
try clustering and see if anythign intersting happens?

In [13]:
### Write your answer here (code cell(s) to be used, as appropriate)

con = sqlite3.connect(sqlf)
dfc = pd.read_sql_query("SELECT * FROM Client;", con, parse_dates=["birthDate"])
con.close()


dfc["birthDate"].describe()

count                             5369
mean     1953-09-12 09:32:21.143602176
min                1911-08-20 00:00:00
25%                1940-11-25 00:00:00
50%                1954-05-06 00:00:00
75%                1968-06-09 00:00:00
max                1987-09-27 00:00:00
Name: birthDate, dtype: object

In [ ]:
# ok let's try this
# todo include whether they paid back loans in the past
# todo include account creation date
sql_read_loan_data = """
SELECT Loan.loanStatus, Loan.loanAmount, Client.birthDate, 
City.inhabitants, City.ratioUrban, City.avgSalary, City.unemployment1995, City.unemployment1996, City.entrepeneurs, City.crimes1995, City.crimes1996
FROM (((Loan
JOIN Disposition ON Loan.dispID = Disposition.dispID)
JOIN Client ON Disposition.clientID = Client.clientID)
JOIN City ON Client.cityID = City.cityID)
WHERE loanStatus IN ('A', 'B');
"""
con = sqlite3.connect(sqlf)
df_risk = pd.read_sql_query(sql_read_loan_data, con, parse_dates=["birthDate"])
con.close()
df_risk

In [88]:
df_risk.dropna(inplace=True)  # one or two empty unemployment fields
y = df_risk["loanStatus"].astype("category")
y = y.cat.codes  # 0 = paid off, 1 = not paid
y

0      0
1      1
2      0
3      0
4      0
      ..
229    0
230    0
231    0
232    0
233    0
Length: 232, dtype: int8

In [70]:
import numpy as np
from sklearn.linear_model import LogisticRegression

In [89]:
# todo train test split
# todo add bank balance

# idea: filter transacitons by credit card type. for customer retention stuff

# well, keep slogging at it and something will come to me even if I can't see it now

X = df_risk.drop(columns=["loanStatus"])
X["birthDate"] = X["birthDate"].dt.year

model = LogisticRegression()
model.fit(X=X, y=y)

LogisticRegression()

----
----

## Task 4: Experimental Results and Analysis (25 Marks)

Given the **five** modelling solutions implemented above, analyse, discuss and present your findings to the key stakeholders of the bank.

Your answer should clearly cover the following:
* Present your findings in a clear and concise manner;
* Discuss your results in the context of the selected solution;
* Discuss how these results can help the bank in performing customer risk assessment and establishing customer retention strategies;
* Present the limitations (if any) of your solutions in a clear and concise manner.

**World Limit**: 500 words. This limit applies only to the explanations. There is no limit on any associated code or figures.

**Write your answer here (text cell(s) to be used, as appropriate)**
![PowerPoint®](think.jpg)

In [ ]:
### Write your answer here (code cell(s) to be used, as appropriate)


----
----

## Task 5: Conclusion (10 Marks)

Given the insights derived from Tasks 1-4, provide a conclusion that clearly covers the following:
* A summary of the main points;
* A discussion of the significance of your results;
* Any recommendation(s) resulting from your analysis;
* Any overall ethical considerations arising from the data analysis of this business domain.

**World Limit**: 300 words.

**Write your answer here (text cell(s) to be used, as appropriate)**

In summary, discombobulate.

In [ ]:
### Write your answer here (code cell(s) to be used, as appropriate)


----
----

## Overall Academic Quality (10 Marks)
10 marks are allocated for the clarity and cohesiveness of your answers (both text and code) across all tasks with appropriate, relevant and effective analysis and presentation of the results.

## Deliverables

You should submit the following to the submission point on the teaching portal:

1. the SQLite database produced in Task 2;
2. the completed Jupyter notebook (both .ipynb and HTML files) that also includes the SQL statements (Task 2), the research design and its implementation (Task 3), and the analysis and presentation of your results (Task 4);
3. any figures or diagrams that are included in your answers in the Jupyter notebook.

For each task where text is required, we have provided guidelines above on the suggested word counts. Exceeding the word count will result in any work beyond the word count being disregarded when assessing.